# How to chain runnables

One point about [LangChain Expression Language](/docs/concepts/#langchain-expression-language) is that any two runnables can be "chained" together into sequences. The output of the previous runnable's `.invoke()` call is passed as input to the next runnable. This can be done using the pipe operator (`|`), or the more explicit `.pipe()` method, which does the same thing.

The resulting [`RunnableSequence`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableSequence.html) is itself a runnable, which means it can be invoked, streamed, or further chained just like any other runnable. Advantages of chaining runnables in this way are efficient streaming (the sequence will stream output as soon as it is available), and debugging and tracing with tools like [LangSmith](/docs/how_to/debugging).

```{=mdx}
import PrerequisiteLinks from "@theme/PrerequisiteLinks";

<PrerequisiteLinks>
- [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language)
- [Prompt templates](/docs/concepts/#prompt-templates)
- [Chat models](/docs/concepts/#chat-models)
- [Output parser](/docs/concepts/#output-parsers)
</PrerequisiteLinks>
```

## The pipe operator

To show off how this works, let's go through an example. We'll walk through a common pattern in LangChain: using a [prompt template](/docs/modules/model_io/prompts/) to format input into a [chat model](/docs/modules/model_io/chat/), and finally converting the chat message output into a string with an [output parser](/docs/modules/model_io/output_parsers/).

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="model"
/>
```

In [ ]:
# | output: false
# | echo: false

%pip install -qU langchain langchain_anthropic

import os
from getpass import getpass
from langchain_anthropic import ChatAnthropic

os.environ["ANTHROPIC_API_KEY"] = getpass()

model = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)

In [2]:
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatAnthropic(model_name="claude-3-haiku-20240307")

chain = prompt | model | StrOutputParser()

Prompts and models are both runnable, and the output type from the prompt call is the same as the input type of the chat model, so we can chain them together. We can then invoke the resulting sequence like any other runnable:

In [3]:
chain.invoke({"topic": "bears"})

"Here's a bear joke for you:\n\nWhy don't bears wear socks? They have bear feet!"

### Coercion

We can even combine this chain with more runnables to create another chain. This may involve some input/output formatting using other types of runnables, depending on the required inputs and outputs of the chain components.

For example, let's say we wanted to compose the joke generating chain with another chain that evaluates whether or not the generated joke was funny.

We would need to be careful with how we format the input into the next chain. In the below example, the dict in the chain is automatically parsed and converted into a [`RunnableParallel`](/docs/expression_language/primitives/parallel), which runs all of its values in parallel and returns a dict with the results.

This happens to be the same format the next prompt template expects. Here it is in action:

In [4]:
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()

composed_chain.invoke({"topic": "bears"})

'I think the "bear with fur coats" joke is a pretty good one! It\'s a simple, silly pun that plays on the idea of bears naturally having fur coats, rather than having to wear raincoats. Puns and wordplay can make for some amusing, lighthearted jokes.\n\nHumor is indeed quite subjective, so it\'s hard to say definitively if a joke is universally funny. But I think this bear coat joke has a good chance of eliciting a chuckle or at least a groan from most people. It\'s the kind of simple, silly joke that can work well.\n\nIf you\'d like to try another bear-themed joke, I\'m happy to take a look and give you my thoughts. But I think the one you came up with is a solid, funny little pun. Nice work coming up with a good bear joke!'

Functions will also be coerced into runnables, so you can add custom logic to your chains too. The below chain results in the same logical flow as before:

In [5]:
composed_chain_with_lambda = (
    chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)

composed_chain_with_lambda.invoke({"topic": "beets"})

'I appreciate the effort and pun-derful wordplay in that beet-themed joke! While puns can be a bit corny, I do find that joke fairly amusing. The play on "root-y" and the down-to-earth nature of beets as vegetables is a clever bit of humor. It\'s not a knee-slapper, but it definitely elicits a chuckle and a groan at the same time - which is the sign of a good pun-based joke in my opinion. I\'d give it a solid 3.5 out of 5 on the humor scale. The beet pun game is strong with this one!'

However, keep in mind that using functions like this may interfere with operations like streaming. See [this section](/docs/expression_language/primitives/functions) for more information.

## The `.pipe()` method

We could also compose the same sequence using the `.pipe()` method. Here's what that looks like:

In [6]:
from langchain_core.runnables import RunnableParallel

composed_chain_with_pipe = (
    RunnableParallel({"joke": chain})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(StrOutputParser())
)

composed_chain_with_pipe.invoke({"topic": "battlestar galactica"})

"I appreciate the Battlestar Galactica-themed joke! While I can't reproduce copyrighted material, I can offer my thoughts on the humor. The joke plays on the recurring theme in the show of the Galactica and its fleet staying one step ahead of the pursuing Cylon forces. The punchline highlights the Galactica's superior evasiveness and maneuverability compared to the Cylon Raiders. For fans of the show, this joke likely lands well as it taps into the core narrative dynamics. Humor is quite subjective, but I think this is a clever and relevant joke for Battlestar Galactica enthusiasts. Let me know if you have any other questions!"

## Next steps

You now know some ways to chain two runnables together.

To learn more, see the other how-to guides on runnables in this section.